In [3]:
from NMR_predict import GPR_NMR

In [4]:
# Define path to xyz files and path to SOAP files

XYZ_directory = '/home/alex/Pt_NMR/data/structures/'

SOAP_directory = '/home/alex/Pt_NMR/data/representations/SOAP/'
APE_RF_directory = '/home/alex/Pt_NMR/data/representations/APE_RF/'

SOAP_params = [2.0, 3, 7] # [rcut, nmax, lmax]
APE_RF_params = [0, 3.0, 1000] # [qmol, rcut, dim]
# Set up SOAP_GPR class
pt_gpr = GPR_NMR(descriptor_path=SOAP_directory, descriptor_type='APE_RF',
                 descriptor_params=APE_RF_params, central_atom='Pt', mode ='write', xyz_path=XYZ_directory, xyz_base='st_')

In [8]:
# Define path to and name of labels

target_name = 'Experimental'
target_path = '/home/alex/Pt_NMR/data/labels/final_data_corrected'

# Use predict function for regression and obtaining CV prediction errors

errors_std = pt_gpr.predict(regressor_type='GPR', kernel_degree=2,
                            target_path=target_path, target_name=target_name,
                            normalize=False, alpha=10.0)

print(errors_std)

RMSE (4-fold CV): 241.80041099555007 [ppm] 23.98429113541213 [ppm] (STDEV)
MAE (4-fold CV): 166.05817567567567 [ppm] 13.3553124341291 [ppm] (STDEV)
(166.05817567567567, 13.3553124341291, 241.80041099555007, 23.98429113541213)
